# Configure Batch AI
Create resources needed for Batch AI, and create the configuration file used by Batch AI utilities.

## Imports

In [ ]:
%load_ext dotenv
import os
import shutil
import json

## Set up
Define parameters used to create resources for Batch AI.

In [ ]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use
selected_subscription = "YOUR_SUBSCRIPTION"
# To contain all resources to be created
resource_group = "RESOURCE_GROUP"         
# The region in which resources will be located
location = "eastus"                     
# The default Batch AI workspace to be created
batchai_workspace = "BATCHAI_WORKSPACE"     
# The Batch AI administrator account login
batchai_login = "BATCHAI_LOGIN"             
# The Batch AI administrator account password
batchai_password = "BATCHAI_PASSWORD"       
# Storage for the data, learning script, and outputs
storage_account = "STORAGE_ACCOUNT"         

In [ ]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use
selected_subscription = "Team Danielle Internal"
# To contain all resources to be created
resource_group = "maboumlb"         
# The region in which resources will be located
location = "eastus"                     
# The default Batch AI workspace to be created
batchai_workspace = "maboumlb"     
# The Batch AI administrator account login
batchai_login = "mabou"             
# The Batch AI administrator account password
batchai_password = "H@lloOldFr1end"       
# Storage for the data, learning script, and outputs
storage_account = "maboumlb"         

Read in the parameters.

In [ ]:
%dotenv -o

## Azure account login
The commands below will ensure you are logged in to your Azure account. If you are not logged in, it will initiate a login by popping up a browser at a login page where you will select your Azure account.

In [ ]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

Set the selected subscription as the default.

In [ ]:
%%bash
az account set --subscription "$selected_subscription"
az account show --o table

You will need to register the Batch and Batch AI resources for your subscription. This may take a few seconds to become effective.

In [ ]:
%%bash
az provider register -n Microsoft.Batch
az provider register -n Microsoft.BatchAI

You assign to Batch AI the role of Active Directory Network Contributor so it can manage the cluster for you. The assignee ```9fcb3732-5f52-4135-8c08-9d4bbaf203ea``` is the service principal of Microsoft Azure BatchAI in public cloud.

In [ ]:
!az role assignment create --role "Network Contributor" --assignee 9fcb3732-5f52-4135-8c08-9d4bbaf203ea

## Batch AI configuration
Create the resource group if it doesn't already exist.

In [ ]:
!az group create -l $location -n $resource_group -o table

Create the storage account. If it doesn't already exist, this step may take many minutes.

In [ ]:
!az storage account create -l $location -g $resource_group -n $storage_account -o table

Get a key to the storage account.

In [ ]:
storage_keys_json = !az storage account keys list -g $resource_group -n $storage_account
storage_keys = json.loads(''.join(storage_keys_json))

Create a service principal for Batch AI and configure its access to Azure. Capture and parse the output for later use.

In [ ]:
aad_json = !az ad sp create-for-rbac
aad = json.loads(''.join(aad_json[aad_json.index('{'):]))

Get the information for the selected Azure subscription.

In [ ]:
account_json = !az account show
account = json.loads(''.join(account_json))

Use the above information to create a configuration dictionary used by various Batch AI utilities.

In [ ]:
cfg = {
    "subscription_id": account['id'],
    "aad_client_id": aad['appId'],
    "aad_secret": aad['password'],
    "aad_tenant": aad['tenant'],
    "location": os.getenv('location'),
    "base_url": "",
    "resource_group": os.getenv('resource_group'),
    "workspace": os.getenv('batchai_workspace'),
    "storage_account" : {
        "name": os.getenv('storage_account'),
        "key": storage_keys[0]['value']
    },
    "admin_user" : {
        "name": os.getenv('batchai_login'),
        "password": os.getenv('batchai_password'),
        "ssh_public_key": ""
    },
    "keyvault_id": "",
    "container_registry" : {
        "user": "",
        "password": "",
        "secret_url": ""
    }
}

Write the configuration to a JSON file.

In [ ]:
configuration_path = 'configuration.json'
with open(configuration_path, 'w') as fp:
    json.dump(cfg, fp, indent=4, sort_keys=True)

Next, we will tune hyperparameters using [random search](03_Random_Search.ipynb) using the Batch AI framework.